In [ ]:
# 노트북 공통: 경로와 파일 이름만 여기서 바꾸면 됩니다.
from pathlib import Path

# BASE_DIR = Path(r"D:\3rdFloor\344_SmartBulb_coding\sentilight\data\data_prep")   # Desktop at my office
# BASE_DIR = Path(r"D:\3rdFloor\344_COShow_coding\sentilight\data\data_prep")      # LG gram notebook
BASE_DIR = Path(r"/home/sbbaik/sentilight_project/sentilight_llm/qwen_finetune")   # Legion Ubuntu notebook

TRAIN_JSONL = BASE_DIR / "train_74k.jsonl"   #train.jsonl
VAL_JSONL   = BASE_DIR / "val_74k.jsonl"     #val.jsonl

# 출력(체크포인트) 저장 경로
OUT_DIR = BASE_DIR / "qwen_sentilight_lora"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print(TRAIN_JSONL)
print(VAL_JSONL)
print(OUT_DIR)


In [ ]:
import json
from datasets import Dataset

SYSTEM_MSG = "당신은 조명 제어를 위한 어시스턴트입니다. 반드시 JSON만 출력하세요."
INSTR_MSG  = "다음 문장의 감정에 어울리는 H,S,B,Dimmer,CT 값을 예측하세요. JSON으로만 답하세요."

def load_jsonl(path):
    recs = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            recs.append(json.loads(line))
    return recs

def build_prompt(example):
    # example = {"instruction":..., "input":..., "output":{...}}
    return (
        f"<|im_start|>system\n{SYSTEM_MSG}<|im_end|>\n"
        f"<|im_start|>user\n{INSTR_MSG}\n\n문장: {example['input']}<|im_end|>\n"
        f"<|im_start|>assistant\n{json.dumps(example['output'], ensure_ascii=False)}<|im_end|>\n"
    )

train_raw = load_jsonl(TRAIN_JSONL)
val_raw   = load_jsonl(VAL_JSONL)

train_ds = Dataset.from_list([{"text": build_prompt(e)} for e in train_raw])
val_ds   = Dataset.from_list([{"text": build_prompt(e)} for e in val_raw])

len(train_ds), len(val_ds)


In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tok_fn(batch):
    return tokenizer(
        batch["text"],
        max_length=384,
        truncation=True,
        padding="max_length",
    )

train_tok = train_ds.map(tok_fn, batched=True, remove_columns=["text"])
val_tok   = val_ds.map(tok_fn,   batched=True, remove_columns=["text"])

train_tok[0].keys()


In [ ]:
import torch
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

# RTX 4080은 bfloat16을 지원하므로, 혼합 정밀도 학습을 위해 설정
# 만약 BF16을 지원하지 않는 GPU라면 (예: 10, 20 시리즈 일부) float16을 사용해야 함
TORCH_DTYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_properties(0).major >= 8 else torch.float16

# Load model on GPU
model = None
try:
    print(f"[info] Loading model with {TORCH_DTYPE} on GPU...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=TORCH_DTYPE,  # GPU에 최적화된 dtype 사용
        device_map="auto" # GPU 사용을 위해 'auto'로 설정 (또는 'cuda:0')
    )
    # LoRA 사용 시에는 requires_grad=False인 레이어를 float32로 유지하지 않도록 설정하는 것이 좋음
    model.gradient_checkpointing_enable()
    print(f"[info] {TORCH_DTYPE} model loaded successfully (GPU)")
except Exception as e:
    print(f"[error] Failed to load model: {e}")
    print("Ensure PyTorch, transformers, and CUDA are correctly installed and configured.")
    raise

peft_cfg = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.1,
    bias="none", task_type="CAUSAL_LM",
    target_modules=["q_proj","k_proj","v_proj","o_proj"]
)
model = get_peft_model(model, peft_cfg)
print("[info] LoRA configuration applied successfully")

In [ ]:
import torch
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from trl import SFTTrainer

# RTX 4080은 bfloat16을 지원하므로 bf16=True를 사용
# bf16=True와 fp16=True를 동시에 True로 설정하면 안 됩니다.
use_bf16 = torch.cuda.is_available() and torch.cuda.get_device_properties(0).major >= 8

args = TrainingArguments(
    output_dir=str(OUT_DIR),
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    num_train_epochs=3,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=20,
    save_steps=100,
    save_total_limit=2,
    # === GPU 최적화 수정 사항 ===
    fp16=False,             # bfloat16 사용 시 fp16은 False (동시 사용 불가)
    bf16=use_bf16,          # RTX 4080 (Ampere+)에 최적인 bfloat16 활성화
    # ==========================
    weight_decay=0.05,
    report_to="none",
    remove_unused_columns=False,
)

# causal LM용: labels = input_ids 자동 생성
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    data_collator=data_collator,
)

# 학습 시작
trainer.train()

# 평가 및 저장
metrics = trainer.evaluate()
print("Final eval:", metrics)

trainer.model.save_pretrained(str(OUT_DIR))
tokenizer.save_pretrained(str(OUT_DIR))
print("Saved to:", OUT_DIR)

In [ ]:
#### version check for SFTTrainer()
import inspect
from trl import SFTTrainer
print(inspect.signature(SFTTrainer.__init__))


In [ ]:
##### version checking
import sys, transformers, trl, inspect
print(sys.executable)
print("transformers:", transformers.__version__, "trl:", trl.__version__)
from transformers import TrainingArguments
print(inspect.signature(TrainingArguments.__init__))


In [ ]:
import re, json
from statistics import mean

SYSTEM_MSG = "당신은 조명 제어를 위한 어시스턴트입니다. 반드시 JSON만 출력하세요."
INSTR_MSG  = "다음 문장의 감정에 어울리는 H,S,B,Dimmer,CT 값을 예측하세요. JSON으로만 답하세요."

def clamp(v, lo, hi):
    try:
        v = int(round(float(v)))
    except Exception:
        v = lo
    return max(lo, min(v, hi))

def predict(model, tokenizer, text: str, max_new_tokens=64):
    prompt = (
        f"<|im_start|>system\n{SYSTEM_MSG}<|im_end|>\n"
        f"<|im_start|>user\n{INSTR_MSG}\n\n문장: {text}<|im_end|>\n"
        f"<|im_start|>assistant\n"
    )
    inputs = tokenizer(prompt, return_tensors="pt")
    
    # === GPU 최적화 수정 사항: 입력 텐서를 모델이 있는 GPU로 이동 ===
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    # ==============================================================

    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=64, do_sample=False)
    decoded = tokenizer.decode(out[0], skip_special_tokens=True)
    resp = decoded.split("<|im_start|>assistant")[-1]
    m = re.search(r"\{.*\}", resp, re.DOTALL)
    obj = json.loads(m.group(0)) if m else {}
    obj = {
        "h":      clamp(obj.get("h", 0),   0, 360),
        "s":      clamp(obj.get("s", 0),   0, 100),
        "b":      clamp(obj.get("b", 0),   0, 100),
        "dimmer": clamp(obj.get("dimmer", 0), 0, 100),
        "ct":     clamp(obj.get("ct", 300), 150, 500),
    }
    return obj

# 검증 MAE
val_pred_errs = {"h":[], "s":[], "b":[], "dimmer":[], "ct":[]}
for ex in val_raw:
    y = ex["output"]
    yhat = predict(model, tokenizer, ex["input"])
    for k in val_pred_errs:
        val_pred_errs[k].append(abs(int(yhat[k]) - int(y[k])))

mae = {k: round(mean(v), 2) for k, v in val_pred_errs.items()}
mae

In [ ]:
# 1. 예제 문장 추론
result1 = predict(model, tokenizer, "오늘은 괜히 설레고 들떠요. 상쾌한 기분이에요.")
print("설렘/상쾌 추론 결과:", result1)

result2 = predict(model, tokenizer, "기분이 좋습니다.")
print("기분 좋음 추론 결과:", result2)

In [ ]:
# 선택: 별도 PC 서버에서 서비스할 때
%pip -q install -U fastapi uvicorn

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Req(BaseModel):
    text: str

@app.post("/predict")
def _predict(req: Req):
    return predict(model, tokenizer, req.text)

# uvicorn 실행 예
# uvicorn.run(app, host="0.0.0.0", port=8000)
